In [1]:
def correlation_matrix(X, y, threshold=0.1):
    """Filtre les colonnes numériques basées sur leur corrélation avec la cible."""
    correlations = {}
    for col in X.select_dtypes(include=[np.number]).columns:
        corr = np.corrcoef(X[col], y)[0, 1]
        correlations[col] = corr
    selected = [col for col, corr in correlations.items() if abs(corr) > threshold]
    print(f"Colonnes numériques retenues (corrélation > {threshold}): {selected}")
    return selected

def categorical_analysis(X, y, threshold=0.2):
    """Filtre les colonnes catégorielles en se basant sur la variance des moyennes."""
    X_with_target = X.copy()
    X_with_target['target'] = y

    selected = []
    for col in X.select_dtypes(include='object').columns:
        means = X_with_target.groupby(col)['target'].mean()
        if means.var() > threshold:
            selected.append(col)
    print(f"Colonnes catégorielles retenues (variance > {threshold}): {selected}")
    return selected

def low_variance_filter(X, threshold=0.01):
    """Filtre les colonnes numériques avec une faible variance."""
    X_numeric = X.select_dtypes(include=[np.number])
    variances = X_numeric.var()
    selected = variances[variances > threshold].index.tolist()
    print(f"Colonnes numériques retenues (variance > {threshold}): {selected}")
    return selected


def auto_handle_nan(df, nan_threshold_delete=0.5, nan_threshold_impute=0.1):
    """
    Traite automatiquement les valeurs NaN dans un dataset.
    - Supprime les colonnes avec trop de NaN.
    - Impute (remplace) les NaN avec des stratégies adaptées :
      - Moyenne pour colonnes numériques.
      - Mode ou "Inconnu" pour colonnes catégorielles.
    """
    print("Analyse des NaN dans le dataset...\n")
    df = df.replace(-1, 1)
    nan_percent = df.isnull().mean()
    print("Pourcentage de valeurs manquantes par colonne :")
    print(nan_percent)
    
    cols_to_delete = nan_percent[nan_percent > nan_threshold_delete].index
    print(f"\nColonnes supprimées (trop de NaN > {nan_threshold_delete*100}%): {list(cols_to_delete)}")
    df = df.drop(columns=cols_to_delete)
    
    for col in df.columns:
        missing = df[col].isnull().sum()
        if missing > 0:
            if df[col].dtype == 'object':
                if nan_percent[col] > nan_threshold_impute:
                    print(f"Colonne '{col}' : Imputation avec 'Manquant' (catégorielle)")
                    df[col] = df[col].fillna("Manquant")
                else:
                    print(f"Colonne '{col}' : Imputation avec la valeur la plus fréquente (mode)")
                    df[col] = df[col].fillna(df[col].mode()[0])
            else:
                if nan_percent[col] > nan_threshold_impute:
                    print(f"Colonne '{col}' : Imputation avec la médiane (numérique)")
                    df[col] = df[col].fillna(df[col].median())
                else:
                    print(f"Colonne '{col}' : Imputation avec la moyenne (numérique)")
                    df[col] = df[col].fillna(df[col].mean())
    
    print("\nTraitement des NaN terminé.")
    return df


In [2]:
import numpy as np

def train_test_split(X, y, test_size=0.2, random_state=None):
    print("Division des données en ensembles d'entraînement et de validation...")
    if random_state:
        np.random.seed(random_state)
    indices = np.arange(len(X))
    np.random.shuffle(indices)

    split_idx = int(len(X) * (1 - test_size))
    train_indices = indices[:split_idx]
    test_indices = indices[split_idx:]

    X = np.array(X)
    y = np.array(y)

    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]



def evaluate_model_with_mapping(model, X_val, y_val, index_to_class):
    # Prédictions en indices
    predictions_indices = model.predict(X_val)

    # Conversion des indices de prédiction et de validation en étiquettes réelles
    predictions = np.array([index_to_class[idx] for idx in predictions_indices])
    y_val_actual = np.array([index_to_class[idx] for idx in y_val])

    # Évaluation avec les vraies étiquettes
    f1 = f1_score_manual(y_val_actual, predictions)
    conf_matrix = confusion_matrix_manual(y_val_actual, predictions)

    return f1, conf_matrix


def f1_score_manual(y_true, y_pred):
    f1_scores = []
    for label in np.unique(y_true):
        tp = np.sum((y_true == label) & (y_pred == label))
        fp = np.sum((y_true != label) & (y_pred == label))
        fn = np.sum((y_true == label) & (y_pred != label))
        if tp + fp == 0 or tp + fn == 0:
            f1_scores.append(0)
        else:
            precision = tp / (tp + fp)
            recall = tp / (tp + fn)
            f1_scores.append(2 * (precision * recall) / (precision + recall))
    return np.mean(f1_scores)

def confusion_matrix_manual(y_true, y_pred):
    num_classes = len(np.unique(y_true))
    conf_matrix = np.zeros((num_classes, num_classes), dtype=int)
    for t, p in zip(y_true, y_pred):
        conf_matrix[t, p] += 1
    return conf_matrix

def evaluate_model(model, X_val, y_val):
    predictions = model.predict(X_val)
    print("Prédictions :", predictions[:10])
    print("Vraies valeurs :", y_val[:10])
    f1 = f1_score_manual(y_val, predictions)
    conf_matrix = confusion_matrix_manual(y_val, predictions)
    return f1, conf_matrix
